# Test

In [ ]:
import os 
import sys
cwd = os.getcwd()
sys.path.append(cwd)
import test_data_util

test_data_util.TestClass.setup_class()
threshold = 0.5
test = test_data_util.TestClass()
test.test_generate_bin_mask()
test.test_create_filtered_cooler(threshold)

# os.remove(['test_data_util.cool', f'test_data_util_{thres}filtered.cool'])

# Application

In [ ]:
import data_util
import cooler

output_path = "/home1/yxiao977/sc1/test_func/test_imap_filtered.cool"
clr = cooler.Cooler("/home1/yxiao977/sc1/train_akita/data/5000res_Dplus_hic.cool")

bin_mask = data_util.generate_bin_mask(
    clr, [data_util.cis_total_ratio_filter(thres=0.5)]
)
data_util.create_filtered_cooler(
    output_path, clr, bin_mask, chunksize=10_000_000, nproc=16
)

In [ ]:
import data_util

hic_target_files = ['/home1/yxiao977/sc1/test_func/5000res_Dplus_0.5filtered.mcool::/resolutions/5000']
genomeName = 'dinoflagellate'
output_dir = '/home1/yxiao977/sc1/akita_dinoflagellate/real_data/'

data_util.create_gap_file(output_dir, hic_target_files, genomeName, num_bad_conseq=10)

In [ ]:
! ./akita_data.py -g ~/sc1/test_func/231103_gaps_dinoflagellate_binSize5000_numconseq10.bed --sample 1 -l 250000 --local -o ~/sc1/train_akita/data/4m_unfiltered --as_obsexp -p 8 -t .1 -v .1 -w 5000 --snap 5000 --stride_train 250000 --stride_test 50000 ~/sc1/train_akita/data/GSE152150_Smic1.1N.fa ~/sc1/train_akita/data/unfiltered_cools.txt